In [1]:
%matplotlib inline

In [21]:
from os import walk
import numpy as np
import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.cm as cm

from PIL import Image

import theano
import theano.tensor as T
from theano import shared
from theano.tensor.nnet.neighbours import images2neibs

In [13]:
import os

# This method reads all the images in sorted order of their names and adds them to the Ims array.

def imgRead():
    Ims = np.empty(shape = (213, 256*256), dtype = np.float32);

    for root, dirs, files in walk("./jaffe/", topdown=True):
        files.sort()   
        for i in range(0, len(files)):  
            img = Image.open(os.path.join(root, files[i]))
            Ims[i] = np.array(img).flatten()
    
    
    return Ims

In [42]:
# Untouched Code
# Plots 9 reconstructions of Image by varying the number of eigen vectors used

def plot_mul(c, D, im_num, X_mn, num_coeffs):
    f, axarr = plt.subplots(3, 3)

    for i in range(3):
        for j in range(3):
            nc = num_coeffs[i*3+j]
            cij = c[:nc, im_num]
            Dij = D[:, :nc]
            plot(cij, Dij, X_mn, axarr[i, j])

    f.savefig('output/hw1b_im{0}.png'.format(im_num))
    plt.close(f)

In [43]:
# Plots the image given c, D and X_mn
def plot(c, D, X_mn, ax):    
    P = np.dot(D, c)
    im = Image.fromarray(P.reshape((256,256)) + X_mn)
    ax.imshow(im)

In [44]:
# Plots the top 16 Eigen Vectors 
def plot_top_16(D, sz, imname):
    f, axarr = plt.subplots(4, 4)
    for i in range(4):
        for j in range(4):
            axarr[i,j].imshow(D[:, 4*i + j].reshape(sz, sz), cmap=plt.cm.gray)
    f.savefig(imname)
    plt.close(f)

In [14]:
# Reading Image which are then collapsed into the desired format.
Ims = imgRead()

Ims = Ims.astype(np.float32)
X_mn = np.mean(Ims, 0)
X = Ims - np.repeat(X_mn.reshape(1, -1), Ims.shape[0], 0)

In [23]:
rng = np.random

# Symbolic Variable for current Eigen Vector (d_i)
d = shared(rng.randn(256*256), name="d") 

# Symbolic Variable for Eigen Vectors
D = T.fmatrix('D')

# Symbolic Variable for Eigen Values
L = T.fvector('L') 

# Symbolic Expression for cost function which needed to minimized
cost = T.dot(T.dot(d.T, X.T), T.dot(X, d)) - T.dot(T.dot(D,d).T, (T.dot(D,d)) * L.T)

# Symbolic Expression for finding the gradient of the cost function w.r.t. d
gradient = T.grad(cost, d)

# Theano Function to update the value of d after 1 iteration of the gradient descent algorithm
update = theano.function([D, L], [] , updates=[(d, d + 0.1 * gradient)])

# Theano Function to normalize d after update
norm = theano.function([], [], updates=[(d, d / d.norm(2))])

In [24]:
# Starting values for D and L
eVectors = np.zeros((16, 256*256), dtype = np.float32)
eValues = np.ndarray(16, dtype = np.float32)

for i in range(16):
    t = 0
    while t < 10: # and stopping condition
        update(eVectors[:i, :], eValues[:i])
        norm()
        t += 1
    
    # Set the eigen vector at position i after gradient descent algorithm converges
    eVectors[i] = d.get_value()
    
    # Set the eigen value at position i based on eigen vector computed above
    eValues[i] = np.dot(np.dot(d.get_value().T, X.T), np.dot(X, d.get_value()))

In [27]:
# Encoding the image using the Eigen vectors obtained after gradient descent algorithm
c = np.dot(eVectors, X.T)

In [40]:
def main():
    for i in range(0,200,10):
        plot_mul(c, eVectors.T, i, X_mn.reshape((256, 256)),[1, 2, 4, 6, 8, 10, 12, 14, 16])
        
    plot_top_16(eVectors.T, 256, 'output/hw1b_top16_256.png')

In [41]:
main()